In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
import matplotlib.pyplot as plt

2024-05-16 11:35:59.139931: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-16 11:35:59.143134: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-16 11:35:59.388037: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-16 11:36:00.396144: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 11:36:02.298404: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [8]:
data = []
for filename in os.listdir("/home/allen/CSC 466/RawData/utkcropped"):
    if filename.endswith(".jpg"):
        with Image.open(os.path.join("/home/allen/CSC 466/RawData/utkcropped", filename)):
            row_data = [os.path.join("/home/allen/CSC 466/RawData/utkcropped", filename)]
            filename = filename.split("_")
            if len(filename) > 3:
                # TODO REPLACE INT WITH NAME
                row_data.append(filename[2]) # 3rd integer represents race
                data.append(row_data)

df_image = pd.DataFrame(data, columns = ["filename", "Race"])
df_image["Race"] = df_image["Race"].replace(["2", "3", "4"], "1")

train, test = train_test_split(df_image, test_size=0.2)

train = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_dataframe(
    dataframe = train,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    target_size = (128, 128),
    rescale = 1/255,
    fill_mode = "nearest"
)

test = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_dataframe(
    dataframe = test,
    x_col = "filename",
    y_col = "Race",
    class_mode = "categorical",
    fill_mode = "nearest",
    rescale = 1/255,
    target_size=(128, 128)
)
    
# batch = next(train)
# a = np.array(batch[0])

# img = Image.fromarray(a[0].astype('uint8'), "RGB")
# img.save(f"test.png")
# img.show()

Found 18964 validated image filenames belonging to 2 classes.
Found 4741 validated image filenames belonging to 2 classes.


In [12]:
df_image["Race"].value_counts(normalize=True)

Race
1    0.574858
0    0.425142
Name: proportion, dtype: float64

In [10]:
# compute_class_weight(
#     "balanced", 
#     classes = np.array(["0", "1", "2", "3", "4"]),
#     y = df_image["Race"])

compute_class_weight(
    "balanced", 
    classes = np.array(["0", "1"]),
    y = df_image["Race"])

array([1.1760766 , 0.86978058])

In [22]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(128, 128, 3)))
model.add(keras.layers.Dense(64, activation = "relu"))
model.add(keras.layers.Dense(32, activation = "relu"))
model.add(keras.layers.Dense(2, activation = "sigmoid"))
model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"], optimizer=keras.optimizers.Adam(learning_rate=0.0001))
model.fit(train,
        #   class_weight={0:0.47159069, 1:1.05761299, 2:1.34428332, 3:1.19706978, 4:2.81906433},
          validation_data = test,
          batch_size = 16,
          epochs = 10)

Epoch 1/10


/home/allen/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


593/593 ━━━━━━━━━━━━━━━━━━━━ 27s 43ms/step - accuracy: 0.6141 - loss: 43.1010 - val_accuracy: 0.7296 - val_loss: 13.8673
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.7238 - loss: 13.1154 - val_accuracy: 0.6208 - val_loss: 32.8414
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 27s 44ms/step - accuracy: 0.6974 - loss: 24.8294 - val_accuracy: 0.7399 - val_loss: 13.2519
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 26s 43ms/step - accuracy: 0.7446 - loss: 11.8497 - val_accuracy: 0.6826 - val_loss: 19.8356
Epoch 5/10
191/593 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.7566 - loss: 10.6565

KeyboardInterrupt: 

In [8]:
model = keras.Sequential()
model.add(keras.layers.Conv2D(128, (1, 1), input_shape=(128, 128, 3), activation = "relu"))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(5, activation = "softmax"))


model.compile(loss = "CategoricalCrossentropy", metrics = ["accuracy"])
model.fit(train,
          validation_data = test,
          class_weight={0:0.47159069, 1:1.05761299, 2:1.34428332, 3:1.19706978, 4:2.81906433},
          epochs = 10)

Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 71s 118ms/step - accuracy: 0.2626 - loss: 1760.7875 - val_accuracy: 0.3088 - val_loss: 1.7084
Epoch 2/10
 63/593 ━━━━━━━━━━━━━━━━━━━━ 57s 109ms/step - accuracy: 0.3559 - loss: 1.5754

KeyboardInterrupt: 